# Algoritmo para descargar Imagenes Satelitales  

## Hackathon AiLab 2021

**Equipo 20: Pista Latente ML**
* Andrea Berenice Ek Hobak
* Gabriela Marali Mundo Cortés
* Mario Xavier Canche Uc
* Myrna Citlali Castillo Silva

**Referencias**  

En  https://code.earthengine.google.com/?scriptPath=Examples%3ADatasets%2FCOPERNICUS_S2_SR con la heramienta de selecciónb rectangular se puede obtener las coordenadas de la zona a analizar a la que se le va a a sacar la imagen.  

En http://www.gisandbeers.com/script-para-descarga-masiva-de-imagenes-satelitales/ se puede leer un tutorial de como obtener las coordenadas, también contiene información de los satelites y desde cuando estan capturando imagenes satelitales.

En https://developers.google.com/earth-engine/datasets se puede encontrar información de los satelites y posibles aplicaciones.

En https://colab.research.google.com/github/csaybar/EEwPython/blob/master/10_Export.ipynb#scrollTo=v6M68WksFMZ1 hay ejemplos de como usar la libreria Google Earth Engine

# Para ejecutar en Google Colab en Drive

In [1]:
# Montamos el Drive al Notebook
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# Verificamos el directorio en el que nos encontramos
!pwd
!ls

/content
drive  sample_data


In [3]:
# Cambiamos de directorio al Drive
import os
os.chdir("drive/My Drive/HackathonAiLab2021/Code")
!ls

 cantidad_de_agua.csv		   SegmentaAguaConClustering.ipynb
'Copia de segmentation.ipynb'	   SegmentaAguaConClustering_xavi2.ipynb
 Descarga_ImagenSatelital2.ipynb   SegmentaAgua.ipynb
 Descarga_ImagenSatelital.ipynb    segmentation.ipynb
 images				   Untitled0.ipynb
 images2


# Importamos las librerias

In [4]:
from PIL import Image
import urllib.request

import ee # Google Earth Engine
from os import system

# Conectamos a Google Earth Engine

In [5]:
# Autenticamos la cuenta
ee.Authenticate()

# Inicializamos
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=L85GZUOvdFM65JCJz746t62lmZuB_lLUihSeIEo6Tck&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjgG5pCmtpZR5d0Cs6lkC9asG0O5mHeQ1vfP-6DEVzM3Mg53RlqMTM

Successfully saved authorization token.


# Definimos las funciones a utilizar

In [6]:
%pwd

'/content/drive/My Drive/HackathonAiLab2021/Code'

In [44]:
def sentinel2(start_date, end_date, geometry, imageVisParam):
    #Imagen para SENTINEL 2
    #IMGSentinel= ee.ImageCollection ('COPERNICUS/S2').filterDate(start_date, end_date).filterBounds(geometry).filterMetadata ('CLOUDY_PIXEL_PERCENTAGE', 'Less_Than', 30)
    IMGSentinel = ee.ImageCollection ('COPERNICUS/S2_SR').filterDate(start_date, end_date).filterBounds(geometry).filterMetadata ('CLOUDY_PIXEL_PERCENTAGE', 'Less_Than', 30)
    print("size: ", IMGSentinel.size().getInfo())

    collectionList = IMGSentinel.toList(IMGSentinel.size())
    collectionSize = collectionList.size().getInfo()

    imgs = []
    fechas = []
    for i in range(collectionSize):
        # Tomamos el primero de la coleccion de imagenes
        SentinelFiltro = ee.Image(collectionList.get(i))
        # Recortamos a nuestra region de interes
        SentinelClip = SentinelFiltro.clip(geometry)

        # Preparamos para visualizar en RGB
        imagenRGB = SentinelClip.visualize(**imageVisParam)

        # Convertimos a Imagen
        url = imagenRGB.getThumbURL({'dimensions':2000,'format':'png'})
        print( ee.Date(ee.Image(collectionList.get(i)).get('system:time_start')).format('Y-M-d').getInfo() )

        imgs.append(imagenRGB)
        fechas.append(ee.Date(ee.Image(collectionList.get(i)).get('system:time_start')).format('Y-M-d').getInfo())
    return(imgs,fechas)

    # Tomamos el primero de la coleccion de imagenes
#    SentinelFiltro = ee.Image(IMGSentinel.first())
    # Recortamos a nuestra region de interes
#    SentinelClip = SentinelFiltro.clip(geometry)

    # Preparamos para visualizar en RGB
#    imagenRGB = SentinelClip.visualize(**imageVisParam)
#    return(imagenRGB)

def download_image(list_imagenRGB, rutaImg, extension, view_url=False):

    for i, imagenRGB in enumerate(list_imagenRGB):
        # Convertimos a Imagen
        url = imagenRGB.getThumbURL({'dimensions':2000,'format':extension})
        if view_url:
            print(url)
        # Descargamos la Imagen
        respuesta = urllib.request.urlopen(urllib.parse.quote(url,safe='://'))

        if respuesta:
            image = Image.open(respuesta)
            image.save(rutaImg[i])
            print("Estatus de la descarga: ", True)
        else:
            print("Estatus de la descarga: ", False)

#        return(False)


# Descargamos una imagen de una fecha especifica

In [53]:
# Coordenadas del mapa
geometry = ee.Geometry.MultiPolygon(
        [[[-99.19204416781875, 21.80894073028815],
           [-99.19204416781875, 21.79156777925953],
           [-99.16547956973525, 21.79156777925953],
           [-99.16547956973525, 21.80894073028815]]
         ], None, False)

geometry = ee.Geometry.MultiPolygon(  # Presa de Malpaso, Chiapas
          [[-93.70933768936047, 17.223196572466385],
           [-93.70933768936047, 17.116917524290802],
           [-93.52342841812023, 17.116917524290802],
           [-93.52342841812023, 17.223196572466385]], None, False)

geometry = ee.Geometry.MultiPolygon(  # Presa bacurato, Sinaloa
          [[-107.94750752607435, 25.975069849424408],
           [-107.94750752607435, 25.839498097434074],
           [-107.79198231855482, 25.839498097434074],
           [-107.79198231855482, 25.975069849424408]], None, False)

geometry = ee.Geometry.MultiPolygon(  # La huasteca potosina
          [[-99.33702038452527, 22.100548964973846],
           [-99.33702038452527, 21.715128898981334],
           [-98.81928967163465, 21.715128898981334],
           [-98.81928967163465, 22.100548964973846]]
           , None, False)

geometry = ee.Geometry.MultiPolygon( # Presa La Purisma de Guanajuato, Gto
         [[[-101.31204841323742, 20.915021363160914],
           [-101.31204841323742, 20.8579256308265],
           [-101.24115226455578, 20.8579256308265],
           [-101.24115226455578, 20.915021363160914]]], None, False)

geometry = ee.Geometry.MultiPolygon(  # La huasteca potosina 2
          [[-99.27041577026746, 21.878980133974085],
           [-99.27041577026746, 21.707154707445007],
           [-99.05068920776746, 21.707154707445007],
           [-99.05068920776746, 21.878980133974085]]
           , None, False)

geometry = ee.Geometry.MultiPolygon(  # La huasteca potosina 2
          [[-99.27041577026746, 21.878980133974085],
           [-99.27041577026746, 21.707154707445007],
           [-99.05068920776746, 21.707154707445007],
           [-99.05068920776746, 21.878980133974085]]
           , None, False)

geometry = ee.Geometry.MultiPolygon( # Lago  de Yuriria, Gto
          [[-101.22673270889172, 20.310577751754582],
           [-101.22673270889172, 20.206222227408922],
           [-101.04082343765148, 20.206222227408922],
           [-101.04082343765148, 20.310577751754582]], None, False)

geometry = ee.Geometry.MultiPolygon( # Presa La Purisma de Guanajuato, Gto
          [[-103.57146731786501, 20.52339521529772],
          [-103.57146731786501, 19.986147368658663],
          [-102.98781863622439, 19.986147368658663],
          [-102.98781863622439, 20.52339521529772]], None, False)
dir = "LagoDeChapala"

Bands = ["B1","B2","B3","B4","B5","B6","B7","B8","B8A","B9","B11","B12","WVP","QA60"]
Bands = ["B11"]

# Parametros de la Imagen satelital
#imageVisParam = {"opacity":1,"bands":["B4","B3","B2"],"max":4000,"gamma":1}
#imageVisParam = {"bands":["B4","B3","B2"],"min":0,"max":2000}


In [54]:
ext = 'png'
# Fecha de las imagenes
start_date = '2018-01-01'
end_date = '2018-12-28'

for Band in Bands:
    imageVisParam = {"bands":[Band],"min":0,"max":2000}

    # Extraemos la imagen satelital
    img_sat, fechas = sentinel2(start_date, end_date, geometry, imageVisParam)

    # ruta de salida para guardar la imagen
    output_img = ['images3/huasteca_sentinel2_' + Band + '_' + fecha.replace('-','_') + "_"+str(i)+ '.' + ext for i, fecha in enumerate(fechas)]

    # Guardamos la imagen satelital
    download_image(img_sat, output_img, ext, view_url=True)

size:  2
2018-12-24
2018-12-24
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/81537857f6a4329aca62f9f91625d775-6f2745f5bb18b2deec4c530d0cced698:getPixels
Estatus de la descarga:  True
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/f121884238f30b3e8c0a694d14825986-128dd993c72a60da5b1e8c63f5bc5bc4:getPixels
Estatus de la descarga:  True


# Descargamos meses completos por año

In [71]:
# Coordenadas del mapa
geometry = ee.Geometry.MultiPolygon(  # Presa de Malpaso, Chiapas
          [[-93.70933768936047, 17.223196572466385],
           [-93.70933768936047, 17.116917524290802],
           [-93.52342841812023, 17.116917524290802],
           [-93.52342841812023, 17.223196572466385]], None, False)
#dir = "PresaMalpasoChiapas"

geometry = ee.Geometry.MultiPolygon(  # Presa bacurato, Sinaloa
          [[-107.94750752607435, 25.975069849424408],
           [-107.94750752607435, 25.839498097434074],
           [-107.79198231855482, 25.839498097434074],
           [-107.79198231855482, 25.975069849424408]], None, False)

geometry = ee.Geometry.MultiPolygon(  # La huasteca potosina
          [[-99.33702038452527, 22.100548964973846],
           [-99.33702038452527, 21.715128898981334],
           [-98.81928967163465, 21.715128898981334],
           [-98.81928967163465, 22.100548964973846]]
           , None, False)
#dir = "HuastecaPotosina"

#geometry = ee.Geometry.MultiPolygon(  # La huasteca potosina 2
#          [[-99.27041577026746, 21.878980133974085],
#           [-99.27041577026746, 21.707154707445007],
#           [-99.05068920776746, 21.707154707445007],
#           [-99.05068920776746, 21.878980133974085]]
#           , None, False)
#dir = "HuastecaPotosina2"

geometry = ee.Geometry.MultiPolygon( # Lago  de Yuriria, Gto
          [[-101.22673270889172, 20.310577751754582],
           [-101.22673270889172, 20.206222227408922],
           [-101.04082343765148, 20.206222227408922],
           [-101.04082343765148, 20.310577751754582]], None, False)
dir = "LagoYuririaGto_Color"

geometry = ee.Geometry.MultiPolygon( # Presa La Purisma de Guanajuato, Gto
         [[[-101.31204841323742, 20.915021363160914],
           [-101.31204841323742, 20.8579256308265],
           [-101.24115226455578, 20.8579256308265],
           [-101.24115226455578, 20.915021363160914]]], None, False)
dir = "PresaLaPurisimaGto_Color"

geometry = ee.Geometry.MultiPolygon( # Lago de Chapala
          [[-103.57146731786501, 20.52339521529772],
          [-103.57146731786501, 19.986147368658663],
          [-102.98781863622439, 19.986147368658663],
          [-102.98781863622439, 20.52339521529772]], None, False)
dir = "LagoDeChapala"

#geometry = ee.Geometry.MultiPolygon( # Laguna Janitzio
#          [[-101.76284305028689, 19.725234283935716],
#          [-101.76284305028689, 19.49885058650945],
#          [-101.50603763036501, 19.49885058650945],
#          [-101.50603763036501, 19.725234283935716]], None, False)
#dir = "Janitzio_Color"

#geometry = ee.Geometry.MultiPolygon( # Laguna de Catemaco
#[[-95.12057773046267, 18.47819779051724],
#          [-95.12057773046267, 18.332256983742415],
#          [-94.96573916845095, 18.332256983742415],
#          [-94.96573916845095, 18.47819779051724]], None, False)
#dir = "LagunaDeCatemaco_Color"

# Parametros de la Imagen satelital
#imageVisParam = {"opacity":1,"bands":["B4","B3","B2"],"max":2000,"gamma":1}
#imageVisParam = {"opacity":1,"bands":["B4","B3","B2"],"max":2000,"gamma":1}

#imageVisParam = {"bands":["B4","B3","B2"],"min":0,"max":2000}
imageVisParam = {"bands":["B11"],"min":0,"max":2000}

In [72]:
ext = 'png'

# Creamos la carpeta
os.system("mkdir -p images3/"+dir)

# Recorremos los años
for year in ['2015','2016','2017','2018','2019','2020','2021']:
    # Recorremos los meses
    for i in range(1,13):
        # Fecha de las imagenes
        start_date = year + '-' + str(i).zfill(2) + '-01'
        end_date = year + '-' + str(i).zfill(2) + '-28'

        # ruta de salida para guardar la imagen
        #output_img = 'images2/'+dir+'/sentinel2_' + start_date.replace('-','_') +  '.' + ext

        try:
            # Extraemos la imagen satelital
            img_sat, fechas = sentinel2(start_date, end_date, geometry, imageVisParam)

            # ruta de salida para guardar la imagen
            output_img = ['images3/'+dir+'/sentinel2_' + Band + '_' + fecha.replace('-','_') + "_"+str(i)+ '.' + ext for i, fecha in enumerate(fechas)]

            # Guardamos la imagen satelital
            status = download_image(img_sat, output_img, ext)
            #print(year, i, " Estatus de la descarga: ", status)
        except:
            print("No se encontro información para: ", start_date)

size:  0
No se encontro información para:  2015-01-01
size:  0
No se encontro información para:  2015-02-01
size:  0
No se encontro información para:  2015-03-01
size:  0
No se encontro información para:  2015-04-01
size:  0
No se encontro información para:  2015-05-01
size:  0
No se encontro información para:  2015-06-01
size:  0
No se encontro información para:  2015-07-01
size:  0
No se encontro información para:  2015-08-01
size:  0
No se encontro información para:  2015-09-01
size:  0
No se encontro información para:  2015-10-01
size:  0
No se encontro información para:  2015-11-01
size:  0
No se encontro información para:  2015-12-01
size:  0
No se encontro información para:  2016-01-01
size:  0
No se encontro información para:  2016-02-01
size:  0
No se encontro información para:  2016-03-01
size:  0
No se encontro información para:  2016-04-01
size:  0
No se encontro información para:  2016-05-01
size:  0
No se encontro información para:  2016-06-01
size:  0
No se encontro info